#### Abreviaturas:
- df: data frame
- cg_coef: coeficientes de Clebsch Gordan para SU(3)
- dtot: número de degeneraciones totales

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Se importa el archivo
df = pd.read_csv('8x8comp.txt', skiprows=4, header=None)

# Se agregan las cabeceras de las columnas
df.columns = ['klm', 'k1l1m1', 'k2l2m2', 'cg_coef', 'multiplet', 'degeneracy', 'dtot']

# Se eliminan las filas con valores NaN
df = df.dropna()

# Convierte las columnas "degeneracy" y "dtot" en tipo int
df = df.astype({"degeneracy":int})
df = df.astype({"dtot":int})

# Retorna una lista con los multipletes existentes
multiplets = df['multiplet'].value_counts().index.tolist() 

In [3]:
# Función para separar el df en multipletes
def divide_df(multiplet_i, dataf, headerc):
    df_multiplet = dataf[dataf[headerc] == multiplet_i]
    return df_multiplet

In [4]:
from sympy import *

In [5]:
# Calcula el cgc para un caso específico dentro de un dfe previamente filtrado
def cgc_md(initial1, initial2, final, ddf):
    dff = ddf.query('k1l1m1 == @initial1').query('k2l2m2 == @initial2').query('klm == @final')
    cgc_str=dff.loc[:,'cg_coef'].values[0]
    cgc = parse_expr(cgc_str, evaluate=0)
    return cgc

# Verifica si el df filtrado es simétrico o antisimétrico
def is_symmetric(sdf):
    p1 = sdf['k1l1m1'].head(1).values[0]
    p2 = sdf['k2l2m2'].head(1).values[0]
    pf = sdf['klm'].head(1).values[0]
    val1 = cgc_md(p1, p2, pf, sdf)
    val2 = cgc_md(p2, p1, pf, sdf)
    if val1 == val2:
        return 'S'
    else:
        return 'A'

In [6]:
Symmetry = []
Multiplets = []
Degeneracy = []
Index = []
for i in range(len(multiplets)):
    # Retorna un df del multiplete indicado en la función
    m = divide_df(multiplets[i], df, 'multiplet')
    
    # Retorna una lista con las degeneraciones del multiplete m
    degeneracy = m['degeneracy'].value_counts().index.tolist()
    
    if len(degeneracy)>1:
        for j in range(len(degeneracy)):
            Multiplets.append(multiplets[i])
            
            # Divide el multiplete en df por degeneraciones
            d = divide_df(degeneracy[j], m, 'degeneracy')
            
            Index.append(d.index)
            Symmetry.append(is_symmetric(d))
            Degeneracy.append(degeneracy[j])
    else:
        Index.append(m.index)
        Multiplets.append(multiplets[i])
        Symmetry.append(is_symmetric(m))
        Degeneracy.append(degeneracy[0])

# Elabora un df para describir cada multiplete
df_describe = pd.DataFrame()
df_describe['multiplet'] = Multiplets
df_describe['degeneracy'] = Degeneracy
df_describe['is_symm'] = Symmetry
df_describe

,multiplet,degeneracy,is_symm
0,11,2,S
1,11,1,A
2,22,1,S
3,30,1,A
4,03,1,A
5,00,1,S


Para extraser los valores de la columna del multiplete usar:
~~~
df_describe['multiplet'].values #array
multiplets #lista
~~~

In [7]:
# Agrega una columna extra al df original
extra =[]
for i in range(df.shape[0]):
    extra.append('x')
df.loc[:,'is_symm']=extra
# Asigna correctamente la característica si es simétrico o antisimétrico
for i in range(len(Index)):
    for j in Index[i]:
        df.at[Index[i],'is_symm']=df_describe.loc[i, 'is_symm']
df

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
0,000,100,212,sqrt(1/8),00,1,1,S
1,000,101,211,-sqrt(1/8),00,1,1,S
2,000,111,111,-sqrt(1/8),00,1,1,S
3,000,200,202,sqrt(1/8),00,1,1,S
4,000,201,201,-sqrt(1/8),00,1,1,S
...,...,...,...,...,...,...,...,...
303,301,211,201,-sqrt(1/3),30,1,1,A
304,301,212,200,-sqrt(1/6),30,1,1,A
305,302,211,202,-sqrt(1/6),30,1,1,A
306,302,212,201,-sqrt(1/3),30,1,1,A


### Algoritmo para el cálculo de coeficientes

#### Abrebiaciones
- I Isoespín
- Y Hipercarga
- I3 Tercera componente de Isoespín

In [8]:
# Transforma los números cuánticos de una particula de y, i, iz al tipo k, l, m
def transform(particle):
    # Separa una palabra por caracteres y luego convierte cada elemento a entero
    p = int(list('11')[0])
    q = int(list('11')[1])
    
    # Convierte a caracter lo que convirtio en entero
    k = str(int((p+2*q)/3 + particle[0]/2 + particle[1]))
    l = str(int((p+2*q)/3 + particle[0]/2 - particle[1]))
    m = str(int((p+2*q)/3 + particle[0]/2 + particle[2]))
    
    # Une los caracteres klm en una sola cadena
    klm = ''.join([k, l, m])
    return klm

# Transforma una lista de lista de numeros
def transform_m(listm):
    list_str = []
    for i in range(len(listm)):
        list_str.append(transform(listm[i]))
    return list_str

In [9]:
df.loc[(df['multiplet']=='11')&(df['klm']=='212')&(df['is_symm']=='S')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
154,212,111,212,-sqrt(1/20),11,2,2,S
157,212,201,212,sqrt(3/20),11,2,2,S
158,212,202,211,-sqrt(3/10),11,2,2,S
160,212,212,111,-sqrt(1/20),11,2,2,S
163,212,211,202,-sqrt(3/10),11,2,2,S
164,212,212,201,sqrt(3/20),11,2,2,S


In [10]:
df.loc[(df['klm']=='211')&(df['k1l1m1']=='111')&(df['k2l2m2']=='100')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
176,211,111,100,sqrt(9/20),22,1,1,S


In [11]:
df.loc[(df['multiplet']=='11')&(df['klm']=='212')&(df['is_symm']=='S')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
154,212,111,212,-sqrt(1/20),11,2,2,S
157,212,201,212,sqrt(3/20),11,2,2,S
158,212,202,211,-sqrt(3/10),11,2,2,S
160,212,212,111,-sqrt(1/20),11,2,2,S
163,212,211,202,-sqrt(3/10),11,2,2,S
164,212,212,201,sqrt(3/20),11,2,2,S


In [12]:
df.loc[(df['klm']=='212')&(df['k1l1m1']=='111')&(df['k2l2m2']=='212')&(df['is_symm']=='S')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
154,212,111,212,-sqrt(1/20),11,2,2,S


## TO DO
1. Empezar a formar la matriz de coeficientes de decaimiento con el algoritmo
2. Dp_lambda_p = [cgc(klm, k1l1m1, k2l2m2, octete, S)]

Otra forma de buscar un valor dentro de una columna de un df es:
~~~
df[df['multiplet'].str.contains('00')]
~~~

In [13]:
x=Symbol('x')
d1 = [sqrt(2)*4, x**2/x, Rational(3,4), 'nombre_decaimiento']

In [14]:
Matrix([d1[0:3]])

Matrix([[4*sqrt(2), x, 3/4]])

In [15]:
e = Matrix([1,1])

In [18]:
dimension = (1+e[0])*(1+e[1])*(2+sum(e))/2
if e[0]<e[1]:
    print('dimension ', dimension, '*')
else:
    print('dimension ', dimension)
Imax = (e[0]+e[1])/2
print('Imax ', Imax)
Ymax = (e[0]-e[1])/3
print('Ymax ', Ymax)
Range = 2*Imax +1
print('Range ', Range)

dimension  8
Imax  1
Ymax  0
Range  3


In [19]:
# Retorna las degeneraciones de SU(2) dado un valor de I
def dec_su2(nmax):
    row = []
    
    rowImax = nmax
    size_m = 2*nmax + 1
    for i in range(size_m):
        row.insert(0,nmax)
        nmax = nmax -1
    return row

# Retorna el grupo de partículas pertenecientes al nivel solicitado
def level_particles(level):
    # Retorna la lista de la linea más larga en todo el multiplete
    I3max_line = dec_su2(Imax) # es una lista

    subIsospin3 = []
    subIsospin = []
    subHipercharge = []
    
    # Delimita los valores máximos y mínimos por nivel horizontal
    a = level-1
    b = len(I3max_line)-level + 1
    subIsospin3.append(I3max_line[a:b])
    Imax_line = I3max_line[b-1] # es un número
    list_subI = []
    list_subI.append(Imax_line)
    list_subI = list_subI*(b-a)
    subIsospin.append(list_subI)
    subImax = I3max_line[-level]
    list_subY=[]
    list_subY.append(Ymax)
    list_subY = list_subY*(b-a)
    subHipercharge.append(list_subY)
    
    # Recorre sobre la línea vertical hacia abajo formando grupos de partículas
    prow = []
    Inew = subImax
    Yrow1 = []
    Ynew1 = Ymax
    
    for i in range(e[0]-level+1):
        Inew = nsimplify(Inew -1/2)
        prow.append(Inew)
        Ynew1 = nsimplify(Ynew1 - 1)
        Yrow1.append(Ynew1)
    
    for i in range(len(prow)):
        subIsospin3.append(dec_su2(prow[i]))
        
        len_p = len(I3max_line)-i-1
        list_subI = []
        list_subI.append(prow[i])
        list_subI = list_subI*len_p
        subIsospin.append(list_subI)
        
        list_subY=[]
        list_subY.append(Yrow1[i])
        list_subY = list_subY*len_p
        subHipercharge.append(list_subY)
    
    # Recorre sobre la línea vertical hacia arriba formando grupos de partículas
    qrow = []
    Inew2 = subImax
    Yrow2 = []
    Ynew2 = Ymax
    for i in range(e[1]-level+1):
        Inew2 = nsimplify(Inew2 -1/2)
        qrow.insert(i,Inew2)
        Ynew2 = nsimplify(Ynew2 + 1)
        Yrow2.append(Ynew2)
        
    for i in range(len(qrow)):
        
        subIsospin3.insert(0,dec_su2(qrow[i]))
        list_subI = []
        list_subI.append(qrow[i])
        len_q = len(I3max_line)-i-1
        list_subI = list_subI*len_q
        subIsospin.insert(0,list_subI)
        
        list_subY=[]
        list_subY.append(Yrow2[i])
        list_subY = list_subY*len_q
        subHipercharge.insert(0, list_subY)
    
    #print(subIsospin) 
    #print(subHipercharge)
        
    return subHipercharge, subIsospin, subIsospin3

# Retorna el número de partículas en el grupo del nivel solicitado
def count_particles(levelp):
    tot_particles = 0
    for i in range(len(levelp)):
        tot_particles = tot_particles + len(levelp[i])
    return tot_particles

In [24]:
I3 = []
I = []
Y = []
npart = 0
for i in range(dimension):
    level = i+1
    subY, subI, subI3 = level_particles(level)
    npart = npart + count_particles(subI3)
    I3.append(subI3)
    I.append(subI)
    Y.append(subY)
    
    if npart == dimension:
        break
I3

[[[-1/2, 1/2], [-1, 0, 1], [-1/2, 1/2]], [[0]]]

In [25]:
I

[[[1/2, 1/2], [1, 1, 1], [1/2, 1/2]], [[0]]]

In [26]:
Y

[[[1, 1], [0, 0, 0], [-1, -1]], [[0]]]

In [30]:
Y[0][0][0], I[0][0][0], I3[0][0][0]

(1, 1/2, -1/2)

In [31]:
Y[0][0][0], I[0][0][0], I3[0][0][1]

(1, 1/2, 1/2)